Data QA Case: ``AK_precincts``
========================

Below are the steps involved in performing automated data quality checks on the ``AK_precincts`` shapefile from ``mggg-states``.

This notebook does the following:
1. Collects the following Alaska election data from ``mggg-states``, MEDSL, and Wikipedia:

    - 2016 United States presidential election
    - 2016 United States Senate elections
    - 2016 United States House of Representatives elections
    - 2018 United States Senate elections
    - 2018 United States House of Representatives elections

2. Wrangles the datasets so that they can be compared against each other.
3. Checks if column names in ``AK_precincts`` diverge from the MGGG naming convention (as outlined in ``naming_convention.json``).
4. Compares the vote counts in ``AK_precincts`` with those in the MEDSL and Wikipedia datasets.
5. Prints the aggregated votes in ``AK_precincts`` for ease of spot checking against Secretary of State websites.

*Note:* the automated checks are not completely exhaustive and further manual checks are required.

Automation Check Timestamp: 02:00 pm ET, 14 August 2020

---

After running the automated scripts, we recommend doing the following:

__Data Standardization__

- Manually evaluate column naming discrepancies to determine if changes are needed.
- Manually evaluate column datatypes to determine if changes are needed.

__Data Comparison__

- Manually investigate large differences found through comparing ``AK_precincts`` data with external sources (e.g. Are absentee ballots counted? Are the precinct counts accurate?).
- For overcounts, how are the votes counted? e.g. A `USH__D` count may include votes for all Democratic candidates where external sources may be only counting one main Democratic candidate.
- For more accurate comparisons, compare ``AK_precincts`` data with those in each States' Secretary of State website.

__Topological Soundness__

- Manually examine shapefiles for gaps and overlaps. 
- *Note:* although gaps and overlaps are not necessarily indicators of inaccurate data (because some counties have precinct islands), they *do* mean that the data cannot be for chain runs.

__Data Documentation__

- Do the READMEs provide data sources?
- Do the READMEs describe what aggregation/disaggregation processes were used?
- Do the READMEs discuss discrepancies/caveats in the data?
- Do the READMEs provide scripts used and/or discuss the data wrangling/processing process?

---

Step 0. Setup
----------------

In [13]:
# Install useful Python packages

!pip3 install numpy
!pip3 install pandas
!pip3 install geopandas
!pip3 install wikipedia

!pip3 install git+https://github.com/KeiferC/gdutils.git

  Cloning https://github.com/KeiferC/gdutils.git to /private/var/folders/rd/8_gw6qtn5p727j40493lvfdc0000gn/T/pip-req-build-y0bc1q35
  Created wheel for gdutils: filename=gdutils-0.0.1-py3-none-any.whl size=16498 sha256=e97ecc0bad474f5205b17bd70ebc5dc90ad838afdafabb3641b97df760d65675
  Stored in directory: /private/var/folders/rd/8_gw6qtn5p727j40493lvfdc0000gn/T/pip-ephem-wheel-cache-d27y_vz2/wheels/ea/c0/f6/afea15a1efae0f9d9e257deb636a715adefcbf7f5f49564fc5
Successfully built gdutils


In [14]:
# Import useful Python modules

import numpy as np
import pandas as pd
import geopandas as gpd

import json # for parsing a json file
import wikipedia # unofficial Wikipedia package (wrapper of MediaWiki API)
import os # for ensuring file traversal works regardless of operating system

import gdutils.datamine as dm # data-mining module from gdutils
import gdutils.dataqa as dq # data QA module from gdutils
import gdutils.extract as et # table extraction module from gdutils

from typing import Any, List, Tuple, Dict, Hashable, Union, NoReturn

Step 1. Data collection
---------------------------

__Step 1.1.__ Collect `AK_precincts` data from the `mggg-states`' `AK-shapefiles` GitHub repository.

In [15]:
# Clone 'AK-shapefiles' repository into 'output/mggg/'

# dm.clone_gh_repos(account='mggg-states', 
#                   account_type='orgs', 
#                   repos=['AK-shapefiles'],
#                   outpath=os.path.join('output', 'mggg'))

In [16]:
# Extract a GeoDataFrame from 'AK-shapefiles/AK_precincts.zip'

mggg_gdf = et.read_file(os.path.join('output', 'mggg', 'AK-shapefiles', 
                                     'AK_precincts.zip')).extract()

mggg_gdf.head() # renders first 5 rows of the extracted gf

,ID,AREA,DISTRICT,NAME,POPULATION,USH14D,USH14R,USH14L,PRES16D,PRES16R,...,VAP,WVAP,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,2MORE,geometry
0,266.0,1.553231,01-446,01-446 AURORA,2995.0,336,457,91,295,434,...,2315,1740,92,237,78,2,48,118,229,"POLYGON ((294705.801 1667364.692, 294704.326 1..."
1,329.0,0.578508,01-455,01-455 FAIRBANKS NO. 1,659.0,72,106,16,65,113,...,545,416,16,62,12,0,10,29,36,"POLYGON ((297483.985 1669129.153, 297485.509 1..."
2,267.0,0.469371,01-465,01-465 FAIRBANKS NO. 2,1542.0,108,166,44,120,157,...,1312,853,85,252,37,1,20,64,100,"POLYGON ((297800.944 1668172.899, 297823.138 1..."
3,268.0,0.401854,01-470,01-470 FAIRBANKS NO. 3,1872.0,216,234,54,205,218,...,1531,1047,97,232,36,3,30,86,138,"POLYGON ((296902.053 1668075.791, 296915.198 1..."
4,269.0,0.561294,01-475,01-475 FAIRBANKS NO. 4,1143.0,123,118,40,86,149,...,883,622,28,153,27,0,14,39,106,"POLYGON ((296178.482 1666807.889, 296101.344 1..."


__Step 1.2.__ Gather MEDSL data for comparison purposes.

In [17]:
# Print available MEDSL data to select applicable datasets

# print('{:27} : {}'.format('Repo Name', 'Repo URL'))
# print('------------------------------------------------------------------')

# for (repo, url) in dm.list_gh_repos(account='MEDSL', account_type='orgs'):
#     print("{:27} : {}".format(repo, url))

In [18]:
# Clone applicable MEDSL datasets

medsl_repos = ['official-precinct-returns', # precinct-level 2016 election results
               '2018-elections-official']   # constituency-level 2018 election results

# this will take some time to complete
# dm.clone_gh_repos(account='MEDSL', 
#                   account_type='orgs', 
#                   repos=medsl_repos, 
#                   outpath=os.path.join('output', 'medsl'))

In [19]:
# Find Alaska-specific MEDSL data

# dm.list_files_of_type('.zip', os.path.join('output', 'medsl'))

In [20]:
# Extract DataFrames from:
# 'official-precinct-returns/2016-precinct-president/2016-precinct-president.zip',
# 'output/medsl/official-precinct-returns/2016-precinct-senate/2016-precinct-senate.zip',
# 'offical-precinct-returns/2016-precinct-house/2016-precinct-house.zip', and
# 2018-elections-offical/precinct_2018.zip'

medsl_16_path = os.path.join('output', 'medsl', 'official-precinct-returns')
medsl_18_path = os.path.join('output', 'medsl', '2018-elections-official')

In [21]:
# 2016-precinct-president.zip

medsl_pres16_gdf = et.read_file(os.path.join(medsl_16_path, '2016-precinct-president',
                                             '2016-precinct-president.zip')).extract()

# convert GeoDataFrame to a DataFrame (since don't need 'geometry')
medsl_pres16_df = pd.DataFrame(medsl_pres16_gdf.drop(columns=['geometry']))

medsl_pres16_df.head()

,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,...,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
0,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
1,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,P60012234,"JOHNSON, JOHN FITZGERALD MR.",NaN,NaN,NaN,NaN
2,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,P20003984,"STEIN, JILL",NaN,NaN,NaN,NaN
3,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,P80001571,"TRUMP, DONALD J. / MICHAEL R. PENCE",NaN,NaN,NaN,NaN
4,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# 2016-precinct-senate.zip

medsl_sen16_gdf  = et.read_file(os.path.join(medsl_16_path, '2016-precinct-senate',
                                             '2016-precinct-senate.zip')).extract()
medsl_sen16_df = pd.DataFrame(medsl_sen16_gdf.drop(columns=['geometry']))

medsl_sen16_df.head()

,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,...,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
0,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,S6AL00302,"CRUMPTON, RONALD (RON) STEVEN",NaN,NaN,NaN,NaN
4,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,C.,Richard C. Shelby,NaN,NaN,S6AL00013,NaN,kg:/m/020yj1,300089.0,14659.0,608.0


In [23]:
# 2016-precinct-house.zip

medsl_ush16_gdf  = et.read_file(os.path.join(medsl_16_path, '2016-precinct-house',
                                             '2016-precinct-house.zip')).extract()
medsl_ush16_df = pd.DataFrame(medsl_ush16_gdf.drop(columns=['geometry']))

medsl_ush16_df.head()

,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,...,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
0,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,NaN,NaN,NaN,H6AL02167,"MATHIS, NATHAN",NaN,NaN,NaN,NaN
4,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,...,NaN,Martha Roby,NaN,NaN,H0AL02087,NaN,kg:/m/0drx5mb,412394.0,21192.0,1408.0


In [24]:
# precinct_2018.zip

medsl_18_gdf = et.read_file(os.path.join(medsl_18_path, 'precinct_2018.zip')).extract()
medsl_18_df = pd.DataFrame(medsl_18_gdf.drop(columns=['geometry']))

medsl_18_df.head()

,precinct,office,party,mode,votes,jurisdiction,county,candidate,district,dataverse,year,stage,state,special,writein,state_po,state_fips,state_cen,state_ic
0,10 JONES COMMUNITY CTR,Straight Party,democratic,election day,98,Autauga,Autauga,Alabama Democratic Party,NaN,all,2018,gen,Alabama,False,False,AL,1,63,41
1,10 JONES COMMUNITY CTR,Straight Party,republican,election day,110,Autauga,Autauga,Alabama Republican Party,NaN,all,2018,gen,Alabama,False,False,AL,1,63,41
2,10 JONES COMMUNITY CTR,US House,democratic,election day,118,Autauga,Autauga,Tabitha Isner,2,house,2018,gen,Alabama,False,False,AL,1,63,41
3,10 JONES COMMUNITY CTR,US House,republican,election day,153,Autauga,Autauga,Martha Roby,2,house,2018,gen,Alabama,False,False,AL,1,63,41
4,10 JONES COMMUNITY CTR,US House,NaN,election day,0,Autauga,Autauga,NaN,2,house,2018,gen,Alabama,False,True,AL,1,63,41


__Step 1.3.__ Gather Wikipedia data for comparison purposes.

*Note:* The Wikipedia dataset was compiled from tables scraped from Wikipedia pages. You can review the scraping and wrangling notebook here: [TODO]().

In [25]:
wiki_gdf = et.read_file('wiki_states.csv').extract()
wiki_df = pd.DataFrame(wiki_gdf.drop(columns=['geometry']))

wiki_df.head()

,STATE,PRES16D,PRES16G,PRES16L,PRES16R,SEN16D,SEN16G,SEN16L,SEN16R,USH16D,...,USH17L,USH17R,SEN18D,SEN18G,SEN18L,SEN18R,USH18D,USH18G,USH18L,USH18R
0,ALABAMA,729547.0,9391.0,44467.0,1318255.0,748709.0,NaN,NaN,1335104.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALASKA,116454.0,5735.0,18725.0,163387.0,36200.0,NaN,90825.0,138149.0,111019.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149779.0
2,ARIZONA,1161167.0,34345.0,106327.0,1252401.0,1031245.0,138634.0,NaN,1359267.0,NaN,...,NaN,NaN,1191100.0,57442.0,NaN,1135200.0,NaN,NaN,NaN,NaN
3,ARKANSAS,380494.0,9473.0,29829.0,684872.0,400602.0,NaN,43866.0,661984.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CALIFORNIA,8753788.0,278657.0,478500.0,4483810.0,7542753.0,NaN,NaN,NaN,NaN,...,NaN,NaN,6019422.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Step 2. Data wrangling
---------------------------

*Note:* Only the MEDSL data requires data wrangling for comparison purposes.

__2.1.__ Wrangle MEDSL data.

In [26]:
def pivot_medsl_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a MEDSL DataFrame, return a pivoted DataFrame where the columns 
    are elections and parties and the values are the votes for every precinct.
    
    """
    medsl_pvt = df.pivot_table(index='precinct',
                               columns=['office', 'party'],
                               values='votes')
    medsl_pvt.columns = [' '.join(col).strip() for col in medsl_pvt.columns.values]
    return medsl_pvt

*2.1.1.* Extract and pivot Alaska-specific data from each MEDSL DataFrame.

In [27]:
medsl_pres16_gdf = et.ExtractTable(medsl_pres16_df, column='state', value='Alaska').extract()
medsl_pres16_df = pd.DataFrame(medsl_pres16_gdf.drop(columns=['geometry']))
medsl_pres16_df = pivot_medsl_df(medsl_pres16_df)

medsl_pres16_df.head()

,US President constitution,US President democratic,US President green,US President libertarian,US President new alliance,US President republican
precinct,,,,,,
01-446 Aurora,5.0,295.0,23.0,75.0,7.0,434.0
01-455 Fairbanks No. 1,2.0,65.0,2.0,8.0,1.0,113.0
01-465 Fairbanks No. 2,8.0,120.0,11.0,21.0,1.0,157.0
01-470 Fairbanks No. 3,6.0,205.0,12.0,35.0,2.0,218.0
01-475 Fairbanks No. 4,4.0,86.0,6.0,16.0,2.0,149.0


In [28]:
medsl_sen16_gdf = et.ExtractTable(medsl_sen16_df, column='state', value='Alaska').extract()
medsl_sen16_df = pd.DataFrame(medsl_sen16_gdf.drop(columns=['geometry']))
medsl_sen16_df = pivot_medsl_df(medsl_sen16_df)

medsl_sen16_df.head()

,US Senate democratic,US Senate libertarian,US Senate new alliance,US Senate republican
precinct,,,,
01-446 Aurora,70.0,261.0,39.333333,409.0
01-455 Fairbanks No. 1,20.0,74.0,6.333333,85.0
01-465 Fairbanks No. 2,45.0,108.0,12.000000,129.0
01-470 Fairbanks No. 3,56.0,136.0,26.000000,225.0
01-475 Fairbanks No. 4,28.0,70.0,13.333333,137.0


In [29]:
medsl_ush16_gdf = et.ExtractTable(medsl_ush16_df, column='state', value='Alaska').extract()
medsl_ush16_df = pd.DataFrame(medsl_ush16_gdf.drop(columns=['geometry']))
medsl_ush16_df = pivot_medsl_df(medsl_ush16_df)

medsl_ush16_df.head()

,US House democratic,US House libertarian,US House new alliance,US House republican
precinct,,,,
01-446 Aurora,334.0,109.0,20.0,389.0
01-455 Fairbanks No. 1,62.0,23.0,4.0,106.0
01-465 Fairbanks No. 2,120.0,51.0,7.0,142.0
01-470 Fairbanks No. 3,191.0,48.0,13.0,236.0
01-475 Fairbanks No. 4,104.0,27.0,10.0,133.0


In [30]:
medsl_18_gdf = et.ExtractTable(medsl_18_df, column='state', value='Alaska').extract()
medsl_18_df = pd.DataFrame(medsl_18_gdf.drop(columns=['geometry']))
medsl_18_df = pivot_medsl_df(medsl_18_df)

medsl_18_df.head()

,Governor democratic,Governor independent,Governor libertarian,Governor republican,State House democratic,State House independent,State House libertarian,State House republican,State Senate democratic,State Senate independent,State Senate republican,US House democratic,US House republican
precinct,,,,,,,,,,,,,
01-446 Aurora,399.0,11.0,29.0,427.0,388.0,NaN,NaN,464.0,462.0,NaN,392.0,429.0,419.0
01-455 Fairbanks No. 1,65.0,3.0,3.0,84.0,60.0,NaN,NaN,93.0,75.0,NaN,79.0,66.0,88.0
01-465 Fairbanks No. 2,124.0,4.0,13.0,139.0,139.0,NaN,NaN,137.0,151.0,NaN,127.0,142.0,136.0
01-470 Fairbanks No. 3,253.0,9.0,16.0,203.0,248.0,NaN,NaN,237.0,277.0,NaN,205.0,265.0,223.0
01-475 Fairbanks No. 4,121.0,5.0,9.0,110.0,134.0,NaN,NaN,109.0,136.0,NaN,106.0,136.0,104.0


Step 3. Naming convention checking
--------------------------------------------

__Step 3.1__ Generate naming conventions.

In [32]:
with open('naming_convention.json') as json_file:
    standards_raw = json.load(json_file)
    
offices = dm.get_keys_by_category(standards_raw, 'offices')
parties = dm.get_keys_by_category(standards_raw, 'parties')
counts  = dm.get_keys_by_category(standards_raw, 'counts')
others  = dm.get_keys_by_category(standards_raw, ['geographies', 
                                                  'demographics', 
                                                  'districts', 
                                                  'other'])

elections = [office + format(year, '02') + party 
             for office in offices
             for year in range(0, 21)
             for party in parties 
             if not (office == 'PRES' and year % 4 != 0)]

counts    = [count + format(year, '02') 
             for count in counts 
             for year in range(0, 20)]

standards = elections + counts + others

__Step 3.2.__ Check ``AK_precincts`` compliance with naming conventions.

In [33]:
naming_check = dq.compare_column_names(mggg_gdf, standards)

In [37]:
# Print and store results of naming convention check

(matches, diffs) = naming_check
matched_columns = matches

diffs = list(diffs)
diffs.sort()

print('Discrepancies from naming convention:', diffs)

Discrepancies from naming convention: ['2MORE', 'AMIN', 'AREA', 'ASIAN', 'BLACK', 'DISTRICT', 'ID', 'NAME', 'NHPI', 'OTHER', 'POPULATION', 'PRES16C', 'WHITE']


In [ ]:
Step 4. Compare ``mggg-states`` data with external sources
----------------------------------------------------------